<h2>Objective</h2>

<h3> the main objectuve of this notebook is to make use tranfer learning to make a working protoype model of image classification on the small sample images of plants.</h3>

**evaluation metric**

<u>if the we can make a small dataset from the whole dataset and train any model on basis of it .</u>

then we can use those functions to train various models and find out the best model for our dataset.

In [1]:
# ignoring all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing the necessary libraries

import numpy as np
import pandas as pd
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
# importing the metadata.csv with all the path to images and their labels

tdata=pd.read_csv("C:/Users/WELCOME/Documents/PlantProject-master/data/PlantCLEF2022_trusted_training_metadata.csv",delimiter=';')
# wdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')
# tdata=pd.read_csv('/home/drowsy/Downloads/data/PlantCLEF2022_trusted_training_metadata.csv',delimiter=';')
# wdata=pd.read_csv('/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')

In [16]:
tdata.head(1)

,classid,image_name,image_path,species,genus,family,order,class,source,manual_tag,predicted_tag,predicted_tag_probability,original_url,license,publisher,gbif_occurrence_id,aggregator,dataset_key,image_backup_url,labels
0,2683260,d0749fe4f8ade13dd9402b0f43bc29e8f28af27a.jpg,2683260/d0749fe4f8ade13dd9402b0f43bc29e8f28af2...,Cycas angulata R.Br.,Cycas,Cycadaceae,Cycadales,Cycadopsida,NaN,NaN,habit,0.7386,https://images.ala.org.au/image/proxyImageThum...,http://creativecommons.org/licenses/by/4.0/,ALA species sightings and OzAtlas,1.632965e+09,gbif,84a649ce-ff81-420d-9c41-aa1de59e3766,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,2683260$Cycas angulata R.Br.


In [21]:
print(len(tdata['class'].unique()),
len(tdata['order'].unique()),
len(tdata['family'].unique()),
len(tdata['genus'].unique()),
len(tdata['species'].unique()))

8 84 483 9603 80000


In [4]:
# directory of the images to train the model.

data_dir='C:/Users/WELCOME/Documents/PlantProject-master/data/images'

# converting the path to pathlib for easy anvigation

import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('C:/Users/WELCOME/Documents/PlantProject-master/data/images')

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [5]:
# concating the classid and species of the images
tdata['labels']=tdata['classid'].astype('str')+'$'+tdata['species'].astype('str')

In [22]:
# concating the classid and species of the images
tdata['classlabels']=tdata['classid'].astype('str')+'$'+tdata['class'].astype('str')

In [32]:
labels_df1=tdata['classlabels'].unique()
len(labels_df1)

80000

In [89]:
type(labels_df2[1])

str

In [138]:

# input should be names of columns , 1st should be key and second should be values and 
# 3rd as dataframe with 1 and 2 as columns
def get_labels_on(df,column1,column2):
    df['label']=df[column1].astype('str')+'$'+df[column2].astype('str')
    label_array=df['label'].unique()
    print(label_array)
    label_dict={}
    for label in label_array:
        label_dict.setdefault(str(label).split('$')[0],[]).append(str(label).split('$')[1])
    return label_dict

family_labels=get_labels_on(tdata,'family','classid')

['Cycadaceae$2683260' 'Cycadaceae$2683214' 'Cycadaceae$2683283' ...
 'Schizaeaceae$2650561' 'Schizaeaceae$2650562' 'Schizaeaceae$3744496']


In [142]:
import sys
sys.getsizeof(family_labels)

18520

In [143]:
family_labels

{'Cycadaceae': ['2683260',
  '2683214',
  '2683283',
  '2683262',
  '2683209',
  '2683264',
  '2683259',
  '2683265',
  '2683238',
  '2683293',
  '2683245',
  '2683234',
  '2683252',
  '2683207',
  '2683250',
  '2683254',
  '2683217',
  '2683272',
  '2683297',
  '2683286',
  '2683228',
  '2683266'],
 'Zamiaceae': ['5284011',
  '5284014',
  '2683648',
  '4000158',
  '2683634',
  '4000135',
  '2683629',
  '2683637',
  '2683645',
  '2683652',
  '2683631',
  '2683628',
  '2683650',
  '2683642',
  '2683625',
  '6067581',
  '4000216',
  '2683644',
  '2683499',
  '2683503',
  '2683501',
  '2683516',
  '2683500',
  '2683517',
  '2683495',
  '7729924',
  '2683520',
  '2683722',
  '2683825',
  '2683711',
  '2683784',
  '2683679',
  '2683703',
  '2683828',
  '2683743',
  '2683778',
  '2683713',
  '2683807',
  '2683783',
  '2683748',
  '2683742',
  '2683786',
  '2683759',
  '9848396',
  '2683756',
  '2683827',
  '2683749',
  '2683750',
  '2683776',
  '2683797',
  '2683710',
  '2683794',
  '2683490

In [6]:
labels_df2=tdata['labels'].unique()
type(labels_df2)

numpy.ndarray

In [33]:
# making a dict with classid as key and name of plant species as value

def get_labels(labels):
    label_dict={}
    for i in range(len(labels)):
        label_dict[str(labels[i]).split('$')[0]]=str(labels[i]).split('$')[1]
    return label_dict

plant_all_label_dict=get_labels(labels_df2)
plant_all_label_dict

{'2683260': 'Cycas angulata R.Br.',
 '2683214': 'Cycas armstrongii Miq.',
 '2683283': 'Cycas beddomei Dyer',
 '2683262': 'Cycas calcicola Maconochie',
 '2683209': 'Cycas candida K.D.Hill',
 '2683264': 'Cycas circinalis L.',
 '2683259': 'Cycas clivicola K.D.Hill',
 '2683265': 'Cycas debaoensis Y.C.Zhong & C.J.Chen',
 '2683238': 'Cycas edentata de Laub.',
 '2683293': 'Cycas falcata K.D.Hill',
 '2683245': 'Cycas indica A.Lindstr. & K.D.Hill',
 '2683234': 'Cycas inermis Lour.',
 '2683252': 'Cycas media R.Br.',
 '2683207': 'Cycas micronesica K.D.Hill',
 '2683250': 'Cycas pectinata Buch.-Ham.',
 '2683254': 'Cycas pruinosa Maconochie',
 '2683217': 'Cycas revoluta Thunb.',
 '2683272': 'Cycas rumphii Miq.',
 '2683297': 'Cycas seemannii Braun',
 '2683286': 'Cycas siamensis Miq.',
 '2683228': 'Cycas tropophylla K.D.Hill & P.K.Lôc',
 '2683266': 'Cycas zeylanica A.Lindstr. & K.D.Hill (J.Schust.)',
 '5284011': 'Bowenia serrulata Chamb. (W.Bull)',
 '5284014': 'Bowenia spectabilis Hook. ex Hook.f.',
 

In [37]:
plant_class_labels_dict=get_labels(labels_df1)
plant_class_labels_dict

{'2683260': 'Cycadopsida',
 '2683214': 'Cycadopsida',
 '2683283': 'Cycadopsida',
 '2683262': 'Cycadopsida',
 '2683209': 'Cycadopsida',
 '2683264': 'Cycadopsida',
 '2683259': 'Cycadopsida',
 '2683265': 'Cycadopsida',
 '2683238': 'Cycadopsida',
 '2683293': 'Cycadopsida',
 '2683245': 'Cycadopsida',
 '2683234': 'Cycadopsida',
 '2683252': 'Cycadopsida',
 '2683207': 'Cycadopsida',
 '2683250': 'Cycadopsida',
 '2683254': 'Cycadopsida',
 '2683217': 'Cycadopsida',
 '2683272': 'Cycadopsida',
 '2683297': 'Cycadopsida',
 '2683286': 'Cycadopsida',
 '2683228': 'Cycadopsida',
 '2683266': 'Cycadopsida',
 '5284011': 'Cycadopsida',
 '5284014': 'Cycadopsida',
 '2683648': 'Cycadopsida',
 '4000158': 'Cycadopsida',
 '2683634': 'Cycadopsida',
 '4000135': 'Cycadopsida',
 '2683629': 'Cycadopsida',
 '2683637': 'Cycadopsida',
 '2683645': 'Cycadopsida',
 '2683652': 'Cycadopsida',
 '2683631': 'Cycadopsida',
 '2683628': 'Cycadopsida',
 '2683650': 'Cycadopsida',
 '2683642': 'Cycadopsida',
 '2683625': 'Cycadopsida',
 

make a dict with class names as key and path of image as values
	merge all the keys where classid matches to the same class

In [ ]:
# making a dict with class as key and classid as values


In [ ]:
# a dict with the column1 as key and path to all the given images that come in that key

def get_path(labels,)

In [134]:
# making a dict with name of folder as key 
# and putting the path of all images inside the folder as a list in the dict as values.

def get_class_and_path(l,path):
    labels=list(path.glob('*/*'))[:l]
    images_dict={}
#     labels are the list of all the path of subfolders in directory
#     labels=list(dirc.glob('*'))
    for label in labels:
        key=str(label).split('\\')[-2]
        val=str(label).split('\\')[-1]
        images_dict.setdefault(key,[]).append(val)        
#         putting the name of folder inside main dir in key of dict 
#         putting all the images in the folder in list , in value of dict
    return images_dict
get_class_and_path(19,data_dir)

{'10000918': ['1917353ca15e000d007b2d4bcb97e5e0ef786b23.jpg',
  'b71941601e10b4920d1c8bf123e0ad20a0910b28.jpg'],
 '10003327': ['19c776b02a1d91d38d0e2ebccebd421ae2487467.jpg',
  '234f93fe0a6be1ce13ea733c2865195cbf529a40.jpg',
  '38e1b31a221a901f8c7969bc3d5041531632a365.jpg',
  '4bfdc90b147d723e17ce8bdca2fd6f029ebf43b5.jpg',
  '586042721f9ee7b9b850318fa43dd52e62fe38e5.jpg',
  '642b7a0bdfcfc974b336e57e4ca8f612c4145a82.jpg',
  '691844fc0e3994b6587e02754777bcde835f6863.jpg',
  '6bc169b3330ee30ff18f90f078698f384b20d0ae.jpg',
  '72bc0e12189ca854c6404ab27d44eed59fc33ec6.jpg',
  '832674034e7f4e9fff937c1d8d170ba6092c2a9d.jpg',
  '993b9e631f0ed50556b64fa7be70df4af53b385c.jpg',
  'aab578b4ba7ef192ea262d53137d946ecc26c3fb.jpg',
  'b455e21b43d5e7bf766b32075c077e618bea1513.jpg',
  'db989e2103403b636e2253af18568c9520a8cb8f.jpg',
  'e29eb3c0e2e337f4cd92605a855f31b3c82309bf.jpg'],
 '10006257': ['0203ae9b83724fc311266c66ee2dda50c10dbded.jpg',
  '4c2db6266bb89f5d9df84a691ea310ee4b0862ce.jpg']}

In [135]:
plant_2kimages_dict=get_class_and_path(20,data_dir)

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [195]:
family_labels['Proteaceae'].index('10000918')

385

In [194]:
for key, values in family_labels.items():
    for value in values:
        if '10000918' == value:
            print(key)

Proteaceae


In [175]:
l1=list(family_labels.values())
l1[].index('2683629')

ValueError: '2683629' is not in list

In [218]:

def gen_batch(img_dict,label_dict,path):
    X=[]
    y=[]
    for k_val,i_vals in img_dict.items():
        value = {i for i in label_dict if k_val in label_dict[i]}
        # value2= list(label_dict.keys())[list(label_dict.values()).index(val)]
        print(k_val)
        print(list(value)[0])
        for i_val in i_vals:
            path_val=str(data_dir)+'\\'+k_val+'\\'+i_val
            print(path_val)
            img=cv2.imread(path_val)
            resized_img=cv2.resize(img,(224,224))
            X.append(resized_img)
            y.append(value)
            break
            
gen_batch(plant_2kimages_dict,family_labels,data_dir)

10000918
Proteaceae
C:\Users\WELCOME\Documents\PlantProject-master\data\images\10000918\1917353ca15e000d007b2d4bcb97e5e0ef786b23.jpg
10003327
Proteaceae
C:\Users\WELCOME\Documents\PlantProject-master\data\images\10003327\19c776b02a1d91d38d0e2ebccebd421ae2487467.jpg
10006257
Asteraceae
C:\Users\WELCOME\Documents\PlantProject-master\data\images\10006257\0203ae9b83724fc311266c66ee2dda50c10dbded.jpg
10008016
Heliotropiaceae
C:\Users\WELCOME\Documents\PlantProject-master\data\images\10008016\01f9e8c752eac26b2e8802d79eee8ebcba75a82a.jpg


In [59]:
# processing the images into a list for training set 
# which will be used for training the model

def img_read(image_dict,label_dict):
    x=[]
    y=[]
    sum1=0
    for nameid,images in image_dict.items():
        for image in images:
            img=cv2.imread(str(image))
            resized_img=cv2.resize(img,(224,224))
            x.append(resized_img)
            y.append(label_dict[nameid])
            sum1=sum1+1
            print('number : ',sum1)
        
    x=np.array(x)
    y=np.array(y)
    return x,y

In [1]:
count=0
for i,j in plant_2kimages_dict.items():
    for k in j:
        count+=1
    
count

NameError: name 'plant_2kimages_dict' is not defined

In [61]:
X,y=img_read(plant_2kimages_dict,plant_all_label_dict)

number :  1
number :  2
number :  3
number :  4
number :  5
number :  6
number :  7
number :  8
number :  9
number :  10
number :  11
number :  12
number :  13
number :  14
number :  15
number :  16
number :  17
number :  18
number :  19
number :  20
number :  21
number :  22
number :  23
number :  24
number :  25
number :  26
number :  27
number :  28
number :  29
number :  30
number :  31
number :  32
number :  33
number :  34
number :  35
number :  36
number :  37
number :  38
number :  39
number :  40
number :  41
number :  42
number :  43
number :  44
number :  45
number :  46
number :  47
number :  48
number :  49
number :  50
number :  51
number :  52
number :  53
number :  54
number :  55
number :  56
number :  57
number :  58
number :  59
number :  60
number :  61
number :  62
number :  63
number :  64
number :  65
number :  66
number :  67
number :  68
number :  69
number :  70
number :  71
number :  72
number :  73
number :  74
number :  75
number :  76
number :  77
number :

In [13]:
# from sklearn.model_selection import train_test_split

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [62]:
X=X.reshape(X.shape[0],X.shape[1]*X.shape[2]*X.shape[3])
X.shape

(237, 150528)

In [63]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X=sc.fit_transform(X)

In [64]:
X

array([[-0.90346551, -1.05558464, -1.03191567, ..., -0.96479597,
        -1.28212592, -1.06755114],
       [ 0.04318366, -0.68379757, -0.57410209, ..., -0.75004836,
        -0.83460922, -1.1493889 ],
       [ 0.40040976,  0.02064108, -0.06135088, ...,  0.78622299,
         0.80054798,  0.7983498 ],
       ...,
       [ 0.63260673,  0.02064108, -0.04303834, ...,  0.95141345,
         0.62842617,  0.60193917],
       [ 0.1860741 ,  0.04020882,  0.17671218, ..., -0.86568169,
        -0.86903358, -0.54378947],
       [ 0.5254389 ,  0.76421521,  1.25715222, ...,  0.17501825,
         0.26697037,  0.65104183]])

In [66]:
X=X.reshape(X.shape[0],224,224,3)

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# `pretrained model with input image shape 512x512 , without final layer`

In [41]:
from keras.optimizers import SGD


In [28]:
pre_trained_model=hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
                                 input_shape=(512, 512, 3), trainable=True)


In [39]:
model=keras.models.Sequential([
    pre_trained_model,
    layers.Dense(80000)
])

In [43]:
from tensorflow.keras.metrics import top_k_categorical_accuracy

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.9),
              metrics=[top_k_categorical_accuracy])


In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              207615832 
                                                                 
 dense_1 (Dense)             (None, 80000)             102480000 
                                                                 
Total params: 310,095,832
Trainable params: 309,318,808
Non-trainable params: 777,024
_________________________________________________________________


In [67]:
model1.fit(X,y,verbose=1)

ValueError: in user code:

    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\WELCOME\AppData\Roaming\Python\Python39\site-packages\ke

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(8192, activation='relu'))
model.add(Dense(16000, activation='relu'))
model.add(Dense(80000, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[32000,64000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Mul]

In [50]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the ResNet-50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a dense output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(80000, activation='softmax')(x)

# Create the final model by combining the ResNet-50 base model with the new dense layers
model1 = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base ResNet-50 layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with the Adam optimizer and categorical crossentropy loss
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [12]:
from joblib import load,dump

dump(model,filename='./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib')

['./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib']

In [ ]:
loaded_model=load(filename='model_pre_name.joblib')